#### Cross Check of Outputs in MFLAMPA: <MH/Distr><1D/2D/Time>

##### Step 0: Package

In [1]:
#!/usr/bin/env python3
import os
import re
import numpy as np

##### Step 1: Get the Outputs Ready

##### Step 1(a): Run the test (test_spectra)

In [2]:
#==============================================================================
def run_test(testname):
    
    # Go to the directory in SP/MFLAMPA/
    dirCurr = os.getcwd()
    strdir = "SP/MFLAMPA"
    dirCurrId = dirCurr.index(strdir)
    if(dirCurrId < 0):
        print("Warning: Incorrect directory! Check the path!")
    dirCurr = dirCurr[0:dirCurrId+len(strdir)]
    os.chdir(dirCurr)
    print("Current dir:", os.getcwd())
    
    # Run the test for cross validation
    TypeTestName = type(testname)
    if(TypeTestName != str):
        print("Warning: Incorrect test name! Re-enter the test name!")
    # os.popen("csh -c 'nagfor -xlicinfo'")
    log_test = os.popen(\
        "csh -c 'make test_%s TESTDIR=run_%s'"%(testname, testname))
    print("%s runlog: %s"%(testname, log_test.read()))
#==============================================================================

run_test(testname="spectra")

Current dir: /Users/whliu/test3/SWMF/SP/MFLAMPA


ar: creating archive libMFLAMPA.a
NAG Fortran Compiler Release 7.1(Hanzomon) Build 7138


spectra runlog: /Applications/Xcode.app/Contents/Developer/usr/bin/make test_spectra_compile
./Config.pl -g=20000
Writing new grid size 20000 into src/ModSize.f90 ...
/Applications/Xcode.app/Contents/Developer/usr/bin/make
cd /Users/whliu/test3/SWMF/share/Library/src; /Applications/Xcode.app/Contents/Developer/usr/bin/make LIB
/Applications/Xcode.app/Contents/Developer/usr/bin/make /Users/whliu/test3/SWMF/lib/libSHARE.a
make[4]: `/Users/whliu/test3/SWMF/lib/libSHARE.a' is up to date.

/Users/whliu/test3/SWMF/lib/libSHARE.a has been brought up to date.

cd /Users/whliu/test3/SWMF/util/EMPIRICAL/srcCosmicRay; /Applications/Xcode.app/Contents/Developer/usr/bin/make LIB
make /Users/whliu/test3/SWMF/lib/libEMPIRICALCR.a
make[4]: `/Users/whliu/test3/SWMF/lib/libEMPIRICALCR.a' is up to date.

/Users/whliu/test3/SWMF/lib/libEMPIRICALCR.a has been brought up to date.

cd /Users/whliu/test3/SWMF/util/TIMING/src; /Applications/Xcode.app/Contents/Developer/usr/bin/make LIB

/Users/whliu/test3/SWMF

##### Step 1(b): Load All the Outputs

In [3]:
#==============================================================================
def read_line(filename, linenum):
    # Given filename, read No.{linenum} in this file, and return string
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            return lines[linenum - 1].strip()
    except FileNotFoundError:
        return f"File {filename} not found."
    except IndexError:
        return f"File {filename} does not have {filename} lines."
    except Exception as e:
        return f"An error occurred: {e}"
#==============================================================================
def read_head(filename, lineVars, lineInfo):
    # Given filename, read nVARs and VARs name

    # read VARs/cols name
    VarsName = read_line(filename, lineVars)
    VarsName = VarsName.split(" ")
    for iVar in range(len(VarsName)-1, -1, -1):
        if VarsName[iVar] == "":
            del VarsName[iVar]

    # read file number, simulation time, ncols
    fileInfo = read_line(filename, lineInfo).split(" ")
    for iInfo in range(len(fileInfo)-1, -1, -1):
        if fileInfo[iInfo] == "":
            del fileInfo[iInfo]
    nVars = int(fileInfo[2]) + int(fileInfo[4])

    # Get VARs Name
    VarsName = VarsName[0:nVars]
    return nVars, VarsName
#==============================================================================
def load_outputs(testname):
    # Load all test outputs: <MH/Distr><1D/2D/Time>

    nLon = 4; nLat = 4
    lineMhVars = 5; lineDistrVars = 4; lineInfo = 2
    #==========================================================================
    def read_mh1d(IsPrintFileName=False):
        # Read the MH1D data and turn them into a dictionary

        if len(fileMh1D_I) < 1:
            print("read_mh1d: no files")
            return

        # Get VARs Name and initialize
        filetry = fileMh1D_I[0]
        nVars, VarsName = read_head(path+filetry, lineMhVars, lineInfo)
        dic_mh1d = {}
        for Var in ['Time'] + VarsName:
            dic_mh1d[Var] = [[[] for _ in range(nLat)] for _ in range(nLon)]

        # Read data from files
        for iFile in range(len(fileMh1D_I)):
            fileName = fileMh1D_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)
            iLon = int(fileName[8:11])  # 1-based
            iLat = int(fileName[12:15]) # 1-based

            # Add Time
            fileInfo = read_line(path+fileName, lineInfo).split(" ")
            for iInfo in range(len(fileInfo)-1, -1, -1):
                if fileInfo[iInfo] == "":
                    del fileInfo[iInfo]
            dic_mh1d['Time'][iLon-1][iLat-1].append(float(fileInfo[1]))

            # Add data
            data = np.loadtxt(fname=path+fileName, \
                skiprows=lineMhVars, unpack=True,  \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_mh1d[VarsName[iVar]][iLon-1][iLat-1].append(data[iVar])

        # Return dic_mh1d, structured by
        # Index 1: VAR name
        # Index 2: Longitude
        # Index 3: Latitude
        # Index 4: iIter, at SPTime
        # Index 5 (only for data): LagrID
        print("finish read_mh1d")
        return dic_mh1d
    #==========================================================================
    def read_mh2d(IsPrintFileName=False):
        # Read the MH2D data and turn them into a dictionary

        if len(fileMh2D_I) < 1:
            print("read_mh2d: no files")
            return

        # Get VARs Name and initialize
        filetry = fileMh2D_I[0]
        nVars, VarsName = read_head(path+filetry, lineMhVars, lineInfo)
        dic_mh2d = {}
        for Var in ['Time'] + VarsName:
            dic_mh2d[Var] = []

        # Read data from files
        for iFile in range(len(fileMh2D_I)):
            fileName = fileMh2D_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)

            # Add Time
            fileInfo = read_line(path+fileName, lineInfo).split(" ")
            for iInfo in range(len(fileInfo)-1, -1, -1):
                if fileInfo[iInfo] == "":
                    del fileInfo[iInfo]
            dic_mh2d['Time'].append(float(fileInfo[1]))
            
            # Add data
            data = np.loadtxt(fname=path+fileName, \
                skiprows=lineMhVars, unpack=True,  \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_mh2d[VarsName[iVar]].append(data[iVar])

        # Return dic_mh2d, structured by
        # Index 1: VAR name
        # Index 2: iIter, at SPTime
        # Index 3 (only for data): iLine
        print("finish read_mh2d")
        return dic_mh2d
    #==========================================================================
    def read_mhtime(IsPrintFileName=False):
        # Read the MHTime data and turn them into a dictionary

        if len(fileMhTime_I) < 1:
            print("read_mhtime: no files")
            return

        # Get VARs Name and initialize
        filetry = fileMhTime_I[0]
        nVars, VarsName = read_head(path+filetry, lineMhVars, lineInfo)
        dic_mhtime = {}
        for Var in VarsName:
            dic_mhtime[Var] = [[[] for _ in range(nLat)] for _ in range(nLon)]

        # Read data from files
        for iFile in range(len(fileMhTime_I)):
            fileName = fileMhTime_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)
            iLon = int(fileName[-11:-8]) # 1-based
            iLat = int(fileName[-7:-4])  # 1-based

            # Add data (including time)
            data = np.loadtxt(fname=path+fileName, \
                skiprows=lineMhVars, unpack=True,  \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_mhtime[VarsName[iVar]][iLon-1][iLat-1].append(data[iVar])

        # Return dic_mhtime, structured by
        # Index 1: VAR name
        # Index 2: Longitude
        # Index 3: Latitude
        # Index 4: 0, since we only append once
        # Index 5: iIter, at SPTime
        print("finish read_mhtime")
        return dic_mhtime
    #==========================================================================
    def read_distr1d(fileDistr1D_I, IsPrintFileName=False):
        # Read the Distr1D data and turn them into a dictionary

        if len(fileDistr1D_I) < 1:
            print("read_distr1d: no files")
            return

        # Get VARs Name and initialize
        filetry = fileDistr1D_I[0]
        nVars, VarsName = read_head(path+filetry, lineDistrVars, lineInfo)
        dic_distr1d = {}
        for Var in ['Time'] + VarsName:
            dic_distr1d[Var] = [[[] for _ in range(nLat)] for _ in range(nLon)]

        # Read data from files
        for iFile in range(len(fileDistr1D_I)):
            fileName = fileDistr1D_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)
            iLon = int(fileName[12:15]) # 1-based
            iLat = int(fileName[16:19]) # 1-based

            # Add Time
            fileInfo = read_line(path+fileName, lineInfo).split(" ")
            for iInfo in range(len(fileInfo)-1, -1, -1):
                if fileInfo[iInfo] == "":
                    del fileInfo[iInfo]
            dic_distr1d['Time'][iLon-1][iLat-1].append(float(fileInfo[1]))

            # Read nDim1 and nDim2
            fileDim = read_line(path+fileName, lineInfo+1).split(" ")
            for iInfo in range(len(fileDim)-1, -1, -1):
                if fileDim[iInfo] == "":
                    del fileDim[iInfo]
            nP = int(fileDim[0]); nX = int(fileDim[1])

            # Add data
            data = np.loadtxt(fname=path+fileName,   \
                skiprows=lineDistrVars, unpack=True, \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_distr1d[VarsName[iVar]][iLon-1][iLat-1].append( \
                    data[iVar].reshape(nX, nP))

        # Return dic_distr1d, structured by
        # Index 1: VAR name
        # Index 2: Longitude
        # Index 3: Latitude
        # Index 4: iIter, since we only append once
        # Index 5: LagrID from 1 to iEnd, for the spectrum
        # Index 6: iMomentum/iEnergy, for the spectrum
        print("finish read_distr1d")
        return dic_distr1d
    #==========================================================================
    def read_distr2d(fileDistr2D_I, IsPrintFileName=False):
        # Read the Distr2D data and turn them into a dictionary

        if len(fileDistr2D_I) < 1:
            print("read_distr2d: no files")
            return

        # Get VARs Name and initialize
        filetry = fileDistr2D_I[0]
        nVars, VarsName = read_head(path+filetry, lineDistrVars, lineInfo)
        dic_distr2d = {}
        for Var in ['Time'] + VarsName:
            dic_distr2d[Var] = []

        # Read data from files
        for iFile in range(len(fileDistr2D_I)):
            fileName = fileDistr2D_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)

            # Add Time
            fileInfo = read_line(path+fileName, lineInfo).split(" ")
            for iInfo in range(len(fileInfo)-1, -1, -1):
                if fileInfo[iInfo] == "":
                    del fileInfo[iInfo]
            dic_distr2d['Time'].append(float(fileInfo[1]))

            # Read nDim1 and nDim2
            fileDim = read_line(path+fileName, lineInfo+1).split(" ")
            for iInfo in range(len(fileDim)-1, -1, -1):
                if fileDim[iInfo] == "":
                    del fileDim[iInfo]
            nP = int(fileDim[0]); nX = int(fileDim[1])

            # Add data
            data = np.loadtxt(fname=path+fileName,   \
                skiprows=lineDistrVars, unpack=True, \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_distr2d[VarsName[iVar]].append( \
                    data[iVar].reshape(nX, nP))

        # Return dic_distr2d, structured by
        # Index 1: VAR name
        # Index 2: LagrID, from 1 to iEnd for the spectrum
        # Index 3: iMomentum/iEnergy, for the spectrum
        print("finish read_distr2d")
        return dic_distr2d
    #==========================================================================
    def read_distrtime(fileDistrTime_I, IsPrintFileName=False):
        # Read the DistrTime data and turn them into a dictionary

        if len(fileDistrTime_I) < 1:
            print("read_distrtime: no files")
            return

        # Get VARs Name and initialize
        filetry = fileDistrTime_I[0]
        nVars, VarsName = read_head(path+filetry, lineDistrVars, lineInfo)
        dic_distrtime = {}
        for Var in VarsName:
            dic_distrtime[Var] = \
                [[[] for _ in range(nLat)] for _ in range(nLon)]

        # Read data from files
        for iFile in range(len(fileDistrTime_I)):
            fileName = fileDistrTime_I[iFile]
            if(IsPrintFileName): print("Read:", fileName)
            iLon = int(fileName[-11:-8]) # 1-based
            iLat = int(fileName[-7:-4])  # 1-based

            # Read nDim1 and nDim2
            fileDim = read_line(path+fileName, lineInfo+1).split(" ")
            for iInfo in range(len(fileDim)-1, -1, -1):
                if fileDim[iInfo] == "":
                    del fileDim[iInfo]
            nP = int(fileDim[0]); nTime = int(fileDim[1])

            # Add data (including time)
            data = np.loadtxt(fname=path+fileName,   \
                skiprows=lineDistrVars, unpack=True, \
                usecols=[iVar for iVar in range(nVars)], dtype=float)
            for iVar in range(nVars):
                dic_distrtime[VarsName[iVar]][iLon-1][iLat-1].append( \
                    data[iVar].reshape(nTime, nP))

        # Return dic_distrtime, structured by
        # Index 1: VAR name
        # Index 2: Longitude
        # Index 3: Latitude
        # Index 4: 0, since we only append once
        # Index 5: iIter, for the spectrum at SPTime
        # Index 6: iMomentum/iEnergy, for the spectrum
        print("finish read_distrtime")
        return dic_distrtime
    #==========================================================================

    # Go to the directory in SP/MFLAMPA/
    dirCurr = os.getcwd()
    strdir = "SP/MFLAMPA"
    dirCurrId = dirCurr.index(strdir)
    if(dirCurrId < 0):
        print("Warning: Incorrect directory! Check the path!")
    dirCurr = dirCurr[0:dirCurrId+len(strdir)]
    os.chdir(dirCurr)
    print("Current dir:", os.getcwd())

    # Get the path and all the output file names
    path = "./run_%s/SP/IO2/"%(testname)
    filelist = sorted(os.listdir(path))

    # Declare the arrays the save different types of outputs
    fileMh1D_I   = []; fileDistr1DcdfS_I  = []; fileDistr1DdefR_I = []
    fileMh2D_I   = []; fileDistr2Ddef_I   = []
    fileMhTime_I = []; fileDistrTimedef_I = []
    # Declare the file name format for different types of outputs
    formatMH1D   = re.compile( \
        r"^MH_data_\d{3}_\d{3}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatMH2D   = re.compile( \
        r"^MH_data_R=\d{4}\.\d{2}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatMHTime = re.compile( \
        r"^MH_data_R=\d{4}\.\d{2}_\d{3}_\d{3}.out$")
    formatDistr1DcdfS = re.compile( \
        r"^Distr_cdf_S_\d{3}_\d{3}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatDistr1DdefR = re.compile( \
        r"^Distr_def_R_\d{3}_\d{3}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatDistr2Ddef     = re.compile( \
        r"^Distr_def_R=\d{4}\.\d{2}_.{1}\d{8}_\d{6}_n\d{6}\.out$")
    formatDistrTimedef   = re.compile( \
        r"^Distr_def_R=\d{4}\.\d{2}_\d{3}_\d{3}.out$")

    # Separate different types of outputs to the corresponding lists
    for iFile in range(len(filelist)):
        if formatMH1D.match(filelist[iFile]):
            fileMh1D_I.append(filelist[iFile])
        elif formatMH2D.match(filelist[iFile]):
            fileMh2D_I.append(filelist[iFile])
        elif formatMHTime.match(filelist[iFile]):
            fileMhTime_I.append(filelist[iFile])
        elif formatDistr1DcdfS.match(filelist[iFile]):
            fileDistr1DcdfS_I.append(filelist[iFile])
        elif formatDistr1DdefR.match(filelist[iFile]):
            fileDistr1DdefR_I.append(filelist[iFile])
        elif formatDistr2Ddef.match(filelist[iFile]):
            fileDistr2Ddef_I.append(filelist[iFile])
        elif formatDistrTimedef.match(filelist[iFile]):
            fileDistrTimedef_I.append(filelist[iFile])

    # Read MH data and Distr data
    dic_mh1d   = read_mh1d(IsPrintFileName=False)
    dic_mh2d   = read_mh2d(IsPrintFileName=False)
    dic_mhtime = read_mhtime(IsPrintFileName=False)
    dic_distr1dcdfS  = read_distr1d(fileDistr1DcdfS_I, False)
    dic_distr1ddefR  = read_distr1d(fileDistr1DdefR_I, False)
    dic_distr2ddef   = read_distr2d(fileDistr2Ddef_I, False)
    dic_distrtimedef = read_distrtime(fileDistrTimedef_I, False)

    # Organize all test outputs into one directories
    dicOut = {}
    dicOut['MH1D'] = dic_mh1d
    dicOut['MH2D'] = dic_mh2d
    dicOut['MHTime'] = dic_mhtime
    dicOut['Distr1DcdfS'] = dic_distr1dcdfS
    dicOut['Distr1DdefR'] = dic_distr1ddefR
    dicOut['Distr2Ddef'] = dic_distr2ddef
    dicOut['Distrtimedef'] = dic_distrtimedef
    return dicOut
#==============================================================================

dicSpectra = load_outputs(testname="spectra")

Current dir: /Users/whliu/test3/SWMF/SP/MFLAMPA
finish read_mh1d
finish read_mh2d
finish read_mhtime
finish read_distr1d
finish read_distr1d
finish read_distr2d
finish read_distrtime


##### Step 2: Check MH_data (MH1D, MH2D, MHTime)

In [4]:
#==============================================================================
def check_MHdata(testname):

    pass
#==============================================================================

check_MHdata(testname="spectra")

##### Step 3: Check Distr_data (Distr1D, Distr2D, DistrTime)

In [5]:
#==============================================================================
def check_Distrdata(testname):

    pass
#==============================================================================

check_Distrdata(testname="spectra")